<a href="https://githubtocolab.com/gee-community/geemap/blob/master/docs/notebooks/118_download_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

**Download images directly to a local computer**

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install -U geemap

In [1]:
import os
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-eslamelnahas-jupyter')

In [ ]:
Map = geemap.Map()
Map

## Download a single image

In [ ]:
image = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").first()
Map.addLayer(image, {}, "Landsat")
Map.centerObject(image)

In [ ]:
geemap.download_ee_image(image, "landsat.tif", scale=100)

## Download an image collection

In [ ]:
out_dir = os.path.expanduser("~/Downloads")

In [ ]:
loc = ee.Geometry.Point(-99.2222, 46.7816)
collection = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterBounds(loc)
    .filterDate("2008-01-01", "2020-01-01")
    .filter(ee.Filter.listContains("system:band_names", "N"))
)

In [ ]:
geemap.download_ee_image_collection(collection, out_dir, scale=10)

## Download image by tile

In [3]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-01-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(83.277, 17.7009, 8)
m.add_layer(dataset.mean(), visualization, 'RGB')
m

Map(center=[17.7009, 83.277], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [17]:
out_dir = os.path.expanduser("~/Downloads")

In [5]:
region = ee.Geometry.BBox(83.277,  17.7009, 84.877, 18.9009)
m.centerObject(region)

In [7]:
features = geemap.fishnet(region, rows=2, cols=2,delta=1.0)
m.addLayer(features, {}, "Grids")

In [21]:
image = dataset.mean() 

In [25]:
geemap.download_ee_image_tiles(
   image, features, out_dir, prefix="Sentinel_", crs="EPSG:4326", scale=100
)

Sentinel_1.tif: |                                                     | 0.00/62.1M (raw) [  0.0%] in 00:00 (et…

There is no STAC entry for: None


Sentinel_2.tif: |                                                     | 0.00/62.0M (raw) [  0.0%] in 00:00 (et…

Sentinel_3.tif: |                                                     | 0.00/62.1M (raw) [  0.0%] in 00:00 (et…

Sentinel_4.tif: |                                                     | 0.00/62.1M (raw) [  0.0%] in 00:00 (et…

...

Sentinel_6.tif: |                                                     | 0.00/62.1M (raw) [  0.0%] in 00:00 (et…

Sentinel_7.tif: |                                                     | 0.00/62.1M (raw) [  0.0%] in 00:00 (et…

Sentinel_8.tif: |                                                     | 0.00/62.0M (raw) [  0.0%] in 00:00 (et…

Sentinel_9.tif: |                                                     | 0.00/62.1M (raw) [  0.0%] in 00:00 (et…

Downloaded 9 tiles in 283.9184145927429 seconds.
